In [237]:
import fingertips_py as ftp
from json import loads
import pandas as pd
from urllib.request import urlopen

In [243]:
pd.set_option('display.max_columns', None)

I need to know for each indicator, which area types and year conifgurations exist so that I know which ones I can pick from and minimise computation for data extraction. - Need to see if I can get a list of this

The alternative would be to create a hierarchy of preference and iterate through the data for each hierachy level. If an indicator is found within a level, then it will no longer be queried for further levels. This might still be computationally expensive but needs to be investigated further 

- time period
- sex
- age

# time period, sex and age available for each indicator

# Area types available for each indicator

In [ ]:
raw_available_indicator_at_area = pd.read_json('https://fingertips.phe.org.uk/api/available_data').fillna(value='null')

print(raw_available_indicator_at_area['AreaTypeId'].unique()
      ,len(raw_available_indicator_at_area['AreaTypeId'].unique()),
      len(raw_available_indicator_at_area),
      raw_available_indicator_at_area['IndicatorId'].nunique()
    #   ,
    #   available_indicator_at_area['IndicatorId'].unique()
)

In [ ]:
raw_available_indicator_at_area.columns

In [ ]:
raw_available_indicator_at_area[raw_available_indicator_at_area['IndicatorId']==90813]

# Indicator metadata
To get the name of each indicator

In [ ]:
indicator_metadata = pd.read_csv('https://fingertips.phe.org.uk/api/indicator_metadata/csv/all')

In [ ]:
indicator_metadata['Indicator ID'].nunique()

# Indicator sex, age and timeperiod available

This doesn't actually bring through all of those available in the data. It brings through the settings for the website and what is shown on the website

In [240]:
def df_nested_json(url): 
    with urlopen(url) as response:
        data_str = response.read().decode("utf-8")
        data_json = loads(data_str[1:-1])
    return pd.json_normalize(data_json)

In [241]:
data = df_nested_json('https://fingertips.phe.org.uk/api/grouproot_summaries/by_indicator_id?indicator_ids=90813')

In [244]:
data

,IndicatorName,IID,GroupId,StateSex,StateAge,StateYearRange,TimePeriod,YearRange,Sex.Id,Sex.Name,Age.Id,Age.Name,Age.Sequence,Category.Id,Category.CategoryTypeId,Category.Name,Category.ShortName,DateChanges.HasDataChangedRecently,DateChanges.DateOfLastChange,DataQualityNote.Id,DataQualityNote.Text,DataQualityNote.Comment,Unit.Id,Unit.Value,Unit.Label,ValueType.Id,ValueType.Name
0,Hospital admissions as a result of self-harm (...,90813,1938132923,False,False,False,2021/22,1,4,Persons,305,10-24 yrs,17.0,-1,-1,Undefined,Undefined,False,07 Mar 2023,0,None,None,3,100000.0,"per 100,000",1,Directly standardised rate


# Area Ids metadata
To get the name of the area

In [ ]:
area_ref = pd.read_json('https://fingertips.phe.org.uk/api/area_types').fillna(value='null')

In [ ]:
area_ref.columns

In [ ]:
area_ref

## hierarchy

In [ ]:
area_hierarchy = pd.read_csv('https://docs.google.com/spreadsheets/u/0/d/15RhWWsHPPMLWoxR5sJcpK-vraRkidRY8jsAb_Y_5GwI/gviz/tq?tqx=out:csv&tq&gid=963757659&headers=1')
area_hierarchy = area_hierarchy[['AreaTypeId', 'map_position', 'hierarchy']].dropna()

# Area types available for each indicator with extra metadata

## prepare full table

In [ ]:
available_indicator_at_area = pd.merge(left=raw_available_indicator_at_area, right=area_ref,
         how='left', left_on='AreaTypeId', right_on='Id',
         indicator=True).rename({'_merge': 'area_merge'}, axis=1)

available_indicator_at_area = pd.merge(left=available_indicator_at_area, right=indicator_metadata[['Indicator ID', 'Indicator']],
                                       left_on='IndicatorId', right_on='Indicator ID', how='inner',
                                       indicator=True, )

## create views

In [ ]:
# indicators_per_area = (pd.DataFrame(available_indicator_at_area.groupby(['Name', 'Short', 'Class', 'Sequence','CanBeDisplayedOnMap'])['_merge'].count())
#                        .reset_index()
#                        .sort_values(['Class', 'Sequence'], ascending=False)
#                        .rename({'_merge': 'indicator_count'}, axis=1)
#                        )

In [ ]:
# indicator_at_area_pivot = available_indicator_at_area[['Indicator', 'Name', 'Id']].pivot_table(index='Indicator', columns='Name', aggfunc='count')

## use hierarchy to choose indicators and areas

In [ ]:
full_area_indicator_hierarchy = pd.merge(left=area_hierarchy, right=raw_available_indicator_at_area,
         left_on='AreaTypeId', right_on='AreaTypeId',
         how='inner')

In [ ]:
left = full_area_indicator_hierarchy[full_area_indicator_hierarchy['map_position']=='left']
left['rank'] = left.groupby('IndicatorId')['hierarchy'].rank(method='dense')
keep_left = left[left['rank']==1]


In [ ]:
keep_left['rank'].unique()

In [ ]:
def find_first_hierarchy(data, map_position):
    data = data[data['map_position']==map_position].loc[:, ['IndicatorId','hierarchy','AreaTypeId', 'map_position']]
    data['rank'] = data.groupby('IndicatorId')['hierarchy'].rank(method='dense')
    return data[data['rank']==1].drop('rank', axis=1)

In [ ]:
get_indicator_area = pd.concat([find_first_hierarchy(full_area_indicator_hierarchy, 'left'),
                                find_first_hierarchy(full_area_indicator_hierarchy, 'middle'),
                                find_first_hierarchy(full_area_indicator_hierarchy, 'right')]
)

# Retrieve indicator data

## filter by those that only have all 3 as a test case

In [ ]:
number_of_maps_returned = get_indicator_area.groupby('IndicatorId')['map_position'].count()
ind_ids_keep_3_maps = number_of_maps_returned[number_of_maps_returned==3].index.values

In [ ]:
get_ind_area_combos = get_indicator_area[get_indicator_area['IndicatorId'].isin(ind_ids_keep_3_maps)]

In [ ]:
get_ind_area_combos

## get data

In [ ]:
get_ind_area_combos['AreaTypeId'].unique()

In [ ]:
get_ind_area_combos.groupby('AreaTypeId').count()

In [ ]:
def return_ind_list_for_area_type(data, AreaTypeId):
    return data[data['AreaTypeId']==AreaTypeId]['IndicatorId'].values

In [ ]:
return_ind_list_for_area_type(get_ind_area_combos, 221)

In [ ]:
def get_data(data, area_type_id):
    indicator_list = return_ind_list_for_area_type(data, area_type_id)
    
    replace_comma = '%2C'
    ids_as_str = replace_comma.join([str(i) for i in indicator_list])
    ids_as_str
    data = ftp.retrieve_data.get_data_by_indicator_ids(indicator_ids=ids_as_str, # [Maximum 100]
                                                    area_type_id=area_type_id,
                                                    # parent_area_type_id=15,
                                                    # profile_id=None,
                                                    include_sortable_time_periods=True,
                                                    is_test=False)
    data.to_csv(str(area_type_id)+'.csv')

In [ ]:
# for area_id in get_ind_area_combos['AreaTypeId'].unique():
#     get_data(get_ind_area_combos, area_id)

In [ ]:
pd.read_json('https://fingertips.phe.org.uk/api/indicator_metadata/by_indicator_id?indicator_ids=1204&include_definition=yes&include_system_content=yes')

In [ ]:
# data = ftp.retrieve_data.get_all_data_for_indicators(indicators=[108,
#  113,
#  114,
#  200,
#  212],
#                                                                area_type_id=7,
#                                                                parent_area_type_id=15,
#                                                                filter_by_area_codes=None,
#                                                                is_test=False)

In [ ]:
# # metadata_areas = ftp.metadata.get_all_areas(is_test=False)
# # metadata_value_notes = ftp.metadata.get_all_value_notes(is_test=False)
# # metadata_profiles = ftp.metadata.get_all_profiles(is_test=False)
# # metadata_domains = ftp.metadata.get_domains_in_profile(18)
# metadata_all_indicators = ftp.metadata.get_metadata_for_all_indicators_from_csv(is_test=False)
# metadata_all_indicator_ids = metadata_all_indicators['Indicator ID'].tolist()

In [ ]:
# metadata_all = pd.read_csv('https://fingertips.phe.org.uk/api/indicator_metadata/all?include_definition=yes&include_system_content=yes')